In [26]:
#Importing Libraries
import boto3
import csv
import json
import time

In [27]:
region = 'us-east-2'

#replace username with your name
streamName = 'blossom-data-eng-michaelasante'

In [28]:
# initialized kinesis client
kinesis = boto3.client('kinesis', region_name=region)

In [29]:
# List all active streams
kinesis.list_streams()

{'StreamNames': ['blossom-data-eng-Priscilla',
  'blossom-data-eng-dennis',
  'blossom-data-eng-emily',
  'blossom-data-eng-giftydovie',
  'blossom-data-eng-giftysdovie',
  'blossom-data-eng-michael_asante',
  'blossom-data-eng-richmond',
  'blossom-eng-Priscilla',
  'blossom-eng-juliuslotsu',
  'blossom-eng-michaelasante',
  'blossom-eng-omaj',
  'kojo-kinesis-test',
  'real_estate'],
 'HasMoreStreams': False,
 'ResponseMetadata': {'RequestId': 'e08f56a7-7947-02ec-b6dc-69544e38030f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e08f56a7-7947-02ec-b6dc-69544e38030f',
   'x-amz-id-2': 'adUC3w3knAHjFvVdeamxq06HR1TRUkFkLN+TsaEvdOT93LNCdbybnmeWEKmFwhREzmPe+LQVuCVy8eielWOf1KNX66japRHK',
   'date': 'Sat, 23 Nov 2019 14:09:55 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '374'},
  'RetryAttempts': 0}}

In [30]:
from datetime import datetime
import time
import boto3
from meq_scraper import scrape_meqasa

In [32]:
kinesis = boto3.client('kinesis', region_name='us-east-2')

In [33]:
def get_data(criteria):
    data = scrape_meqasa(1)  # ignore the argument 1
    data = data.head(2) # our assumption; first 2 listings are new
    
    # we may have some criteria to filter our data on eg "address='adenta'"
    if criteria:
        data = data.query(criteria)
    
    # for this exercise, we'll only select these columns
    data = data[['property', 'beds', 'area', 'garages', 'address']]
    
    return data

In [34]:
def listings_producer(stream_name, data):
        response = kinesis.put_record(
            StreamName=stream_name,
            Data=data,
            PartitionKey='blossom',
        )
        if response['ResponseMetadata']['HTTPStatusCode'] == 200:
            print(f"Records pushed to {streamName} within kinesis")
        else:
            print("Records not failed to be pushed to kinesis")

In [35]:
if __name__ == '__main__':
    try:
        while True:
            data = get_data(None)
            data = data.to_json()
            listings_producer('blossom-eng-michaelasante', data)
            time.sleep(10)
            
    except Exception as e:
        print(f"Writing failed. Exiting gracefully due to {e}")

Scraping page: 1 from Meqasa
Records pushed to blossom-data-eng-michaelasante within kinesis
Scraping page: 1 from Meqasa
Records pushed to blossom-data-eng-michaelasante within kinesis
Scraping page: 1 from Meqasa
Records pushed to blossom-data-eng-michaelasante within kinesis
Scraping page: 1 from Meqasa
Records pushed to blossom-data-eng-michaelasante within kinesis
Scraping page: 1 from Meqasa
Records pushed to blossom-data-eng-michaelasante within kinesis
Scraping page: 1 from Meqasa
Writing failed. Exiting gracefully due to HTTPSConnectionPool(host='meqasa.com', port=443): Max retries exceeded with url: /houses-for-rent-in-ghana?w=1 (Caused by SSLError(SSLError("bad handshake: SysCallError(-1, 'Unexpected EOF')")))


In [36]:
import datetime as dt
import json
import time
import boto3
import pandas as pd

In [37]:
kinesis = boto3.client('kinesis', region_name='us-east-2')

In [38]:
def get_shard_iterator(stream_name, shard_id, iterator_type, date):
    shard_response = kinesis.get_shard_iterator(
        StreamName=stream_name, 
        ShardId=shard_id, 
        ShardIteratorType=iterator_type, 
        Timestamp=date
    )

    return shard_response['ShardIterator']

In [40]:
def listings_consumer(next_shard_iterator):
    start = dt.datetime.now()
    finish = start + dt.timedelta(seconds=60)
    
    # we'll only consume for 60 seconds
    while start < finish:
        response = kinesis.get_records(
            ShardIterator=next_shard_iterator,
            Limit=20
        )
        
        try:
            data = response['Records'][0]['Data']
            data = json.loads(data)

            df = pd.DataFrame(data)   
        except IndexError:
            # this error occurs when all records have been consumed so far
            print("No new records have arrived")
            
        # get the next shard iterator ie. where next to get data
        next_shard_iterator = response['NextShardIterator']
        
        # pause checking for new data every 5 seconds
        time.sleep(5)

In [41]:
if __name__ == '__main__':
    stream_name = 'real_estate'
    shard_id = 'shardId-000000000000'
    iterator_type = 'AT_TIMESTAMP'
    date = dt.datetime.today().date().__str__()
    
    next_shard_iterator = get_shard_iterator(stream_name, shard_id, iterator_type, date)
    listings_consumer(next_shard_iterator)

No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have arrived
No new records have 

KeyboardInterrupt: 